# Classes

In [1]:
class Person():
    """A simple class""" 
    class_variable = None # Variable shared between all class instances
    
    def __init__(self, name, title):
        """This gets called to create a new class"""
        local = "hello" # Local variable with scope limited to __init__
        self.name = name 
        self.title = title
        Person.class_variable = title + " " + name
        
    def say_hello(self):
        print("Hello {}".format(self.title))
        
    def cls_variable_example(self):
        print(self.class_variable) # or Person.class_variable

In [2]:
john = Person('John', 'Doctor')
sally = Person('Sally', 'Professor')

In [3]:
john.__dict__

{'name': 'John', 'title': 'Doctor'}

In [4]:
john.name = "Johnny"
john.name

'Johnny'

### A Note about self...
Self is used to describe variables specific to a singular class instance   
so the self.name is specific to the created class instance      
This is why sally and john can have different names stored as shown   
Self is not a reserve keyword, you can use whatever name you like as the first variable    
... but self is used because it's general practice


### Self is what's passed automatically when calling a function from a class instance... 
sally.say_hello() == Person.say_hello(sally) 


In [5]:
sally.say_hello()

Hello Professor


In [6]:
# Call class variable
sally.cls_variable_example()

Professor Sally


In [7]:
# Class variable was last updated when sally was created, hence why this doesn't have John's info
john.cls_variable_example() 

Professor Sally


In [8]:
# Update class_variable for all Person class instances
Person.class_variable = "Shared Between all class instances"

In [9]:
sally.cls_variable_example()
john.cls_variable_example()

Shared Between all class instances
Shared Between all class instances


In [10]:
# Class instances are stored in a dictionary format
sally.__dict__ # or vars(sally) 

{'name': 'Sally', 'title': 'Professor'}

In [11]:
# Class objects are also stored in a dictionary format...
Person.__dict__ # or vars(Person)

mappingproxy({'__module__': '__main__',
              '__doc__': 'A simple class',
              'class_variable': 'Shared Between all class instances',
              '__init__': <function __main__.Person.__init__(self, name, title)>,
              'say_hello': <function __main__.Person.say_hello(self)>,
              'cls_variable_example': <function __main__.Person.cls_variable_example(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>})

# • Able to override python special namespaces with classes to give coherent calling of data...     
Using items like __len__, __eq__, __str__, __bool__, etc... we can override special names in python to keep intuitive coding calls in place   

In [12]:
class Person(object):
    """A simple class""" 
    
    def __init__(self, name, role):
        """This gets called to create a new class"""
        self.name = name
        self.role = role
        
    def __bool__(self):
        if self.name:
            return True
        else:
            return False
 

In [13]:
sally = Person('Sally', 'Engineer')
empty_name = Person('', 'Engineer')

In [14]:
vars(sally)

{'name': 'Sally', 'role': 'Engineer'}

In [15]:
def validate(entity):
    """ Overly verbose function to show output """
    if entity:
        print("Evaluates to True")
    else:
        print("Evaluates to False")

# or just call bool(instance)

In [16]:
validate(sally)
validate(empty_name)

Evaluates to True
Evaluates to False


In [17]:
class Project(object):
    """Class for storing and processing Project records"""

    def __init__(self, name, start, length, cost, members):
        """Init variables from input"""
        self.name = name
        self.start = start
        self.length = length
        self.cost = cost
        self.members = members
        
    def __repr__(self):
        """Used for debug output, but will be used for print if __str__ not defined"""
        return '\n'.join("{}: {}".format(key, value) for key, value in vars(self).items())
    
    def __str__(self):
        """Just as an example..."""
        return self.name
    
    def __len__(self):
        return self.length


In [18]:
import time
project1 = Project("1st Project", time.time(), 21, 1000000, ('NYC', 'CompanyA', 'CompanyB'))
project2 = Project("2st Project", time.time(), 11, 1, ('NYC'))

In [19]:
print(project1)

1st Project


In [20]:
project1 # or just project1 in interactive shell

name: 1st Project
start: 1585856217.17042
length: 21
cost: 1000000
members: ('NYC', 'CompanyA', 'CompanyB')

In [21]:
# being able to call len(class_instance) is much more intuitive than having to know class_instance.length 
# or calling some other random function like project1.length_of_project()
len(project1)

21

In [22]:
project1.costs = 1100000 # Added to class instance on the fly... be careful with typos! 

In [23]:
vars(project1) # cost and costs are now present!

{'name': '1st Project',
 'start': 1585856217.17042,
 'length': 21,
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB'),
 'costs': 1100000}

In [24]:
vars(project2) # No newitem attribute costs added here, all instance specific

{'name': '2st Project',
 'start': 1585856217.17046,
 'length': 11,
 'cost': 1,
 'members': 'NYC'}

# • Slots

In [25]:
# Add slots for performance... treats class like tuple instead of dictionary.. 
import time
class Project_Slots(object):
    """Class for storing and processing Project records"""
    __slots__ = ("name", "start", "length", "cost", "members")

    def __init__(self, name, start, length, cost, members):
        """Init variables from input"""
        self.name = name
        self.start = start
        self.length = length
        self.cost = cost
        self.members = members
        
    def __str__(self):
        """Just as an example..."""
        return self.name


In [26]:
prj_slots = Project_Slots("1st Project", time.time(), 21, 1000000, ('NYC', 'CompanyA', 'CompanyB'))
print(prj_slots)
prj_slots.newitem = 10 # errors due to immutability, can not add new items past what's in the __slots__ 

1st Project


AttributeError: 'Project_Slots' object has no attribute 'newitem'

# • Classmethod

Way to create a class off of arguments other than what's in the init

In [27]:
import time

class Date(object):
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def from_string(cls, string_rep): # cls here is "Date" .. cls can be named anything, cls is similar to self in this regard 
        parts = string_rep.split('-')
        print(parts)
        return cls(int(parts[0]), int(parts[1]), int(parts[2]))

    @classmethod
    def today(cls):
        now = time.localtime()
        return cls(now.tm_year, now.tm_mon, now.tm_mday)

In [28]:
init_ = Date(2010, 11, 5)
vars(init_)

{'year': 2010, 'month': 11, 'day': 5}

In [29]:
string = Date.from_string("2010-12-31")
vars(string)

['2010', '12', '31']


{'year': 2010, 'month': 12, 'day': 31}

In [30]:
today = Date.today()
vars(today)

{'year': 2020, 'month': 4, 'day': 2}

# • Property decorator 

In [31]:
class Project(object):
    """Class for storing and processing Project records"""

    def __init__(self, name, start, length, cost, members):
        """Init variables from input"""
        self.name = name
        self.start = start
        self.length = length
        self.cost = cost
        self.members = members
        
    def __repr__(self):
        """Used for debug output, but will be used for print if __str__ not defined"""
        return '\n'.join("{}: {}".format(key, value) for key, value in vars(self).items())
    
    def __len__(self):
        """Project length in weeks"""
        return self.length
    
    @property
    def cost_per_week(self):
        """Cost per week"""
        return self.cost / self.length 

In [32]:
project1 = Project("1st Project", time.time(), 21, 1000000, ('NYC', 'CompanyA', 'CompanyB'))
vars(project1)

{'name': '1st Project',
 'start': 1585856219.8741388,
 'length': 21,
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB')}

In [33]:
project1.cost_per_week #() no longer needed with property decorator in place, computed on the fly when called 

47619.04761904762

In [34]:
project1.cost_per_week = 100 # no property setter provided yet, this errors

AttributeError: can't set attribute

In [35]:
project1.cost = 20000000 # updates off class instance attributes 
project1.cost_per_week

952380.9523809524

In [36]:
vars(Project)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Class for storing and processing Project records',
              '__init__': <function __main__.Project.__init__(self, name, start, length, cost, members)>,
              '__repr__': <function __main__.Project.__repr__(self)>,
              '__len__': <function __main__.Project.__len__(self)>,
              'cost_per_week': <property at 0x10e43acb0>,
              '__dict__': <attribute '__dict__' of 'Project' objects>,
              '__weakref__': <attribute '__weakref__' of 'Project' objects>})

### Using property variables to check input
Can even be implemented after the fact using the same namespace    

In [37]:
import time
class Project(object):
    """Class for storing and processing Project records"""

    def __init__(self, name, start, length, cost, members):
        """Init variables from input"""
        self.name = name
        self.start = start
        self.length = length
        self.cost = cost
        self.members = members
        
    def __repr__(self):
        """Used for debug output, but will be used for print if __str__ not defined"""
        return '\n'.join("{}: {}".format(key, value) for key, value in vars(self).items())
    
    def __len__(self):
        return self.length
    
    @property 
    def cost_per_week(self):
        return self.cost / self.length 
        
    @property
    def length(self):
        """Add property decorator to enable property setters below"""
        #if not isinstance(self._whatever, int): #example showing how this can be changed
        #    return int(self._whatever)
        return self._whatever # _whatever is an arbitrary variable name, just keep this and below the same 
    
    @length.setter
    def length(self, new_length):
        """"Checks input values off below logic every time variable 'mode' is set"""
        if isinstance(new_length, int) and new_length > 0:
            self._whatever = new_length 
        else:
            raise ValueError("Expecting postive value integer denoting number of weeks for a project")
                 

In [38]:
project1 = Project("1st Project", time.time(), 21, 1000000, ('NYC', 'CompanyA', 'CompanyB'))
vars(project1)

{'name': '1st Project',
 'start': 1585856221.010026,
 '_whatever': 21,
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB')}

In [39]:
project1._whatever = "21"
# project1.__dict__["_whatever"] = "21"
vars(project1)

{'name': '1st Project',
 'start': 1585856221.010026,
 '_whatever': '21',
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB')}

In [40]:
project1.length 


'21'

In [41]:
vars(Project)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Class for storing and processing Project records',
              '__init__': <function __main__.Project.__init__(self, name, start, length, cost, members)>,
              '__repr__': <function __main__.Project.__repr__(self)>,
              '__len__': <function __main__.Project.__len__(self)>,
              'cost_per_week': <property at 0x10e442110>,
              'length': <property at 0x10e4421d0>,
              '__dict__': <attribute '__dict__' of 'Project' objects>,
              '__weakref__': <attribute '__weakref__' of 'Project' objects>})

# • Descriptors

Descriptors provide the underlying magic for most of Python’s class features, including @classmethod, @staticmethod, @property, and even the __slots__ specification    
Descriptors allow for error checking similar to property setters with less code    
Using class variables of another class type (Integer in this case) we create a means to proxy alter the underlying  dictionary of instances of the Records class 

In [42]:
class Integer(object):

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if isinstance(value, int):
            instance.__dict__[self.name] = value
        else:
            raise ValueError("Expecting Integer")

In [43]:
class Records(object): 
    
    nums = Integer('nums')
    nums2 = Integer('nums2')
    
    def __init__(self, record, nums, nums2):
        self.record = record
        self.nums = nums
        self.nums2 = nums2


In [44]:
test = Records("testing", 10, 42) # errors out when a string is provided for nums or nums2
vars(test)

{'record': 'testing', 'nums': 10, 'nums2': 42}

In [45]:
vars(Records)

mappingproxy({'__module__': '__main__',
              'nums': <__main__.Integer at 0x10e443d50>,
              'nums2': <__main__.Integer at 0x10e443d90>,
              '__init__': <function __main__.Records.__init__(self, record, nums, nums2)>,
              '__dict__': <attribute '__dict__' of 'Records' objects>,
              '__weakref__': <attribute '__weakref__' of 'Records' objects>,
              '__doc__': None})

### Descriptors under the covers... starting from the basics...

In [46]:
class Integer(object): 

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            print("inside __get__ from base class")
            return cls.__dict__
        if instance:
            print("inside __get__")
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        print("inside __set__", instance, value)
        instance.__dict__[self.name] = value # comment out to show test instance __dict__ is not updated below


In [47]:
class Records(object): 
    
    nums2 = Integer("_whatevernum2")
    
    def __init__(self, record, nums, nums2):
        self.record = record
        self.nums = nums
        self.nums2 = nums2 # uncomment this to see the nums2 __set__ fire

In [48]:
test = Records("Test", 10, 11)

inside __set__ <__main__.Records object at 0x10e443190> 11


In [49]:
vars(test)

{'record': 'Test', 'nums': 10, '_whatevernum2': 11}

In [50]:
test.nums2

inside __get__


11

In [51]:
test.__dict__["_whatevernum2"] = "12"
test.nums2

inside __get__


'12'

In [52]:
vars(Records)

mappingproxy({'__module__': '__main__',
              'nums2': <__main__.Integer at 0x10e4433d0>,
              '__init__': <function __main__.Records.__init__(self, record, nums, nums2)>,
              '__dict__': <attribute '__dict__' of 'Records' objects>,
              '__weakref__': <attribute '__weakref__' of 'Records' objects>,
              '__doc__': None})

In [53]:
Records.nums2 # call Integer __get__(self, instance, cls): where self is nums2, instance is None, cls is Records

inside __get__ from base class


mappingproxy({'__module__': '__main__',
              'nums2': <__main__.Integer at 0x10e4433d0>,
              '__init__': <function __main__.Records.__init__(self, record, nums, nums2)>,
              '__dict__': <attribute '__dict__' of 'Records' objects>,
              '__weakref__': <attribute '__weakref__' of 'Records' objects>,
              '__doc__': None})

In [54]:
test.nums2 = 10 # call Integer __set__(self, instance, value): where self is nums2, instance is test, value is 10

inside __set__ <__main__.Records object at 0x10e443190> 10


In [55]:
test.nums2 # call Integer __get__(self, instance, cls): where self is nums2, instance is test, cls is Records

inside __get__


10

### Back to the solution... 

In [56]:
class Integer(object):  

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if isinstance(value, int):
            instance.__dict__[self.name] = value
        else:
            raise ValueError("Expecting Int")
                 
class Records(object): 
    
    nums = Integer('_anythingnums')
    nums2 = Integer('nums2')
    
    def __init__(self, record, nums, nums2):
        self.record = record
        self.nums = nums
        self.nums2 = nums2

In [57]:
test = Records("testing", 10, 42)
vars(test)

{'record': 'testing', '_anythingnums': 10, 'nums2': 42}

In [58]:
test.__dict__["_anythingnums"] = "11"
vars(test)

{'record': 'testing', '_anythingnums': '11', 'nums2': 42}

In [59]:
test.nums = "11"

ValueError: Expecting Int

In [60]:
test.__dict__["nums2"] = "42" # again we can get around error checking if we know the correct namespace 
vars(test)

{'record': 'testing', '_anythingnums': '11', 'nums2': '42'}

In [61]:
vars(Records)

mappingproxy({'__module__': '__main__',
              'nums': <__main__.Integer at 0x10a840750>,
              'nums2': <__main__.Integer at 0x10cbab890>,
              '__init__': <function __main__.Records.__init__(self, record, nums, nums2)>,
              '__dict__': <attribute '__dict__' of 'Records' objects>,
              '__weakref__': <attribute '__weakref__' of 'Records' objects>,
              '__doc__': None})

In [62]:
Records.nums2 # Records.nums2.__get__(None, Records)

In [63]:
test.nums2 # Calls Records.nums2.__get__(test, Records)

'42'

In [64]:
# hasattr(Records.__dict__['nums'].__class__, '__set__')
hasattr(Records.nums.__class__, "__set__")

True

In [65]:
Records.nums.__class__.__set__

<function __main__.Integer.__set__(self, instance, value)>

In [66]:
test.__class__.__dict__['nums'].__set__(test, 12)
vars(test)
# test.num = 11

{'record': 'testing', '_anythingnums': 12, 'nums2': '42'}

In [67]:
setattr(test, "nums2", 42)
getattr(test, "nums2")

42

In [68]:
setattr(test, "testing", 42)
vars(test)

{'record': 'testing', '_anythingnums': 12, 'nums2': 42, 'testing': 42}

# Property setters under the covers... quick glance

In [69]:
import time
class Project(object):
    """Class for storing and processing Project records"""

    def __init__(self, name, start, length, cost, members):
        """Init variables from input"""
        self.name = name
        self.start = start
        self.length = length
        self.cost = cost
        self.members = members
        
    def __repr__(self):
        """Used for debug output, but will be used for print if __str__ not defined"""
        return '\n'.join("{}: {}".format(key, value) for key, value in vars(self).items())
    
    def __len__(self):
        return self.length
    
    @property 
    def cost_per_week(self):
        return self.cost / self.length 
        
    @property
    def length(self):
        """Add property decorator to enable property setters below"""
        return self._whatever # _whatever is an arbitrary variable name, just keep this and below the same 
    
    @length.setter
    def length(self, new_length):
        """"Checks input values off below logic every time variable 'mode' is set"""
        if isinstance(new_length, int) and new_length > 0:
            self._whatever = new_length 
        else:
            raise ValueError("Expecting postive value integer denoting number of weeks for a project")
                 

In [70]:
project1 = Project("1st Project", time.time(), 21, 1000000, ('NYC', 'CompanyA', 'CompanyB'))
vars(project1)

{'name': '1st Project',
 'start': 1585856229.610898,
 '_whatever': 21,
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB')}

In [71]:
vars(Project) # same as project1.__class__.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Class for storing and processing Project records',
              '__init__': <function __main__.Project.__init__(self, name, start, length, cost, members)>,
              '__repr__': <function __main__.Project.__repr__(self)>,
              '__len__': <function __main__.Project.__len__(self)>,
              'cost_per_week': <property at 0x10e454d10>,
              'length': <property at 0x10e454830>,
              '__dict__': <attribute '__dict__' of 'Project' objects>,
              '__weakref__': <attribute '__weakref__' of 'Project' objects>})

In [72]:
hasattr(project1.__class__.__dict__['length'], '__set__')
# Same as hasattr(Project.length, "__set__")
# Property has a set method, but is the function passed to by the property decorator "fset" defined? 

True

In [73]:
Project.length.__set__

<method-wrapper '__set__' of property object at 0x10e454830>

In [74]:
# fset is used to handle actual updating of private variable in question 
Project.length.fset

<function __main__.Project.length(self, new_length)>

In [75]:
# cost_per_week as has a __set__ method, but fset and dset (for delete) will not be defined here... only fget
hasattr(Project.__dict__['cost_per_week'], '__set__')

True

In [76]:
bool(Project.cost_per_week.fset)

False

In [77]:
# project1.__class__.__dict__['length'].__set__(project1, 120)
setattr(project1, "length", 120)
vars(project1)

{'name': '1st Project',
 'start': 1585856229.610898,
 '_whatever': 120,
 'cost': 1000000,
 'members': ('NYC', 'CompanyA', 'CompanyB')}